# Observation Targets Assessor
This notebook aims to injest a list of source descriptions and observation criteria from the STM effort, applying appropriate metric tools to assess against the critera for a list of mission concept descriptions.

To run this on Google Colab:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnGBaker/GWI-metrics/blob/main/Notebooks/ObservationTargetsAssessor.ipynb)

### Some basic setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/JohnGBaker/GWI-metrics.git
    src='GWI-metrics/src/'
    datadir='GWI-metrics/Data/'
else:
    src='../src/'
    datadir='../Data/'
!mkdir -p '../plots/'

# Import
import os
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(src)
import constants
import metrics
import sources
import concepts
import subsystems
import background
import pandas as pd
from glob import glob
import re
from astropy.coordinates import Angle
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u

### Define the set of concepts
Here we load in a set of the pre-defined concepts, or you can define your own

In [ ]:
missionNames = (
    'LISACBE',
    #'LISASciRDv1',
    'LISAGrande',
    'LISAU',
    'TwinLISA',
    'GoBIGLISA',
    'ALIA',
    'ALIAtwin',
    'GoBIGALIA')
    #'ALIAlowL')
missions=[concepts.menu[mission] for mission in missionNames]

for mission in missions:
    mission=background.add2model(mission)
#model = concepts.LISACBE.copy()
#modelName = model.get('label')
#model = background.add2model(model)     # add galactic background model

### Load the observation target data
Load information about the target sources and the observation quality targets from the STM.
The data are from https://nasa.sharepoint.com/:x:/t/GravitationalWaveImager/EaaeMC7L-2NJpFCQloYbSnoBDZag_cvFWR5_BjRoFAD6Tw?e=uWr4mU

In [ ]:
#datadir='../Data/'
files=glob(datadir+'ObservationObjectives*.csv')
files.sort(key=os.path.getmtime)
print('Observation targets files are:',files)
file=files[-1]
print('Most recent observation targets file is:',file)
#Read file
df=pd.read_csv(file,header=1)
print(len(df),'rows read from file.')
if True:
    #Drop empty rows
    for i,row in df.iterrows():
        if row.isnull().all():
            df=df.drop(i)
    #Fill empties in first column with values above
    val=float('nan')
    col='Source class'
    for i,row in df.iterrows():
        if row.isnull()[col]:
            df.loc[i,col]=val
        else:
            val=df.loc[i,col]
#Drop rows without a labeled objective
for i,row in df.iterrows():
    if row.isnull()['provisional POC label']:
        df=df.drop(i)
print(len(df),'rows with a potential observation label.')
#Fill empties with value above
if True:
    #for columnName, columnData in df.iteritems():
    for columnName, columnData in df.items():
        val=float('nan')
        nulls=columnData.isnull()
        for i,row in df.iterrows():
            if nulls[i]:
                df.loc[i,columnName]=val
            else:
                val=df.loc[i,columnName]
#derive masses
if True:
    col=2
    df['m1']=float('nan')
    df['m2']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        mtxt=re.search(r"\s*Msun\s*\+\s*.*Msun",txt)
        #print('::',mtxt)
        mtxt=re.search(r"(\d+\.*\d*[eE]*[\+-]*\d*)\s*M[Ss]un\s*\+\s*(\d+\.*\d*[eE]*[\+-]*\d*)\s*M[Ss]un",txt)
        #print(':',mtxt)
        m1=None
        if mtxt is not None: 
            try:
                m1=float(mtxt[1])
                #print(m1)
                m2=float(mtxt[2])
                #print(m2)
            except: 
                pass
        if m1 is None:
            df=df.drop(i)
        else:
            df.loc[i,'m1']=m1
            df.loc[i,'m2']=m2
    print(len(df),'rows with masses.')
#derive redshifts/distances
if True:
    col=3
    df['z']=float('nan')
    df['dist']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        ztxt=re.search(r"z\s*=\s*(\d+\.*\d*[eE]*[\+-]*\d*)",txt)
        z=None
        if ztxt is not None: 
            try:
                z=float(ztxt[1])
                dist=cosmo.luminosity_distance(z).to(u.kpc).value
            except: 
                pass
        if z is None:
            df=df.drop(i)
        else:
            df.loc[i,'z']=z
            df.loc[i,'dist']=dist
    print(len(df),'rows with redshift.')
#derive timecuts
if True:
    col=4
    df['timecut']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        ztxt=re.search(r"(.*)\s+(before|<)\s+merger",txt)
        z=None
        
        if ztxt is not None:
            try:
                ztxt=ztxt[1]
                if ztxt=='month to week': z=15
                elif ztxt=='year to month': z=180
                elif ztxt=='days': z=3
                elif ztxt=='1 day': z=1
                elif ztxt=='week': z=7
                elif ztxt=='2 years': z=360
            except: 
                pass
        df.loc[i,'timecut']=z
#derive SNRs -- assume None by default
if True:
    col=4
    df['SNR']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        ztxt=re.search(r"SNR\s*=\s*(\d+\.*\d*[eE]*[\+-]*\d*)",txt)
        z=None
        if ztxt is not None: 
            try:
                z=float(ztxt[1])
            except: 
                pass
        df.loc[i,'SNR']=z
#prep CW sources
if True:
    col=0
    df['CW']=False
    df['f0']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        if 'Persistent' in txt: 
            df.loc[i,'CW']=True
            #Also need to f0 (or separation)
            txt=row['freq']
            try:
                ftxt=re.search(r"(\d+\.*\d*[eE]*[\+-]*\d*)\s+Hz",txt)[1]
                f0=float(ftxt)
                df.loc[i,'f0']=f0
            except:
                print('Failed to get freq from txt "'+txt+'"')
                df=df.drop(i)
#process angular precision
if True:
    col=6
    df['resolution']=float('nan')
    for i,row in df.iterrows():
        txt=row.iloc[col]
        z=None
        #print(txt)
        try:
            #print("quoted resoltion requirement is: ",txt)
            #print(Angle(txt))
            z=Angle(txt).arcsecond #what's this doing?
            #print("... converted to: ",z)
        except: 
            pass
        df.loc[i,'resolution']=z
#Drop cases with neither angular precision requirement nor SNR
for i,row in df.iterrows():
    if np.isnan(row['resolution']) and np.isnan(row['SNR']):df=df.drop(i)
print(len(df),'rows with constraints (precision and/or SNR).')        
#display(df[20:]) # don't know why we're displaying all but the first 20 entries -- turn off

    

In [ ]:
minidf=df[['Sub-specs','timecut']]
display(minidf.drop_duplicates())

In [ ]:
# make source dictionaries
objectiveList=[]
minimumSNR=5
for i,row in df.iterrows():
    obj={}
    obj['label']=row['provisional POC label']
    obj['m1']=row['m1']*(1+row['z'])#convert to redshifted mass
    obj['m2']=row['m2']*(1+row['z'])#convert to redshifted mass
    dist=cosmo.luminosity_distance(row['z']).to(u.kpc).value
    obj['dl']=dist
    if row['CW']:
        obj['type']='CW'
        obj['f0']=row['f0']
    else:
        obj['type']='chirp'
        if np.isfinite(row['timecut']):
            obj['timecut']=row['timecut']
    #Now add the conditions
    conditions={}
    if np.isfinite(row['resolution']):
        conditions['resolution']=row['resolution']/3600/180*np.pi #convert arcsec to rad
    snr=minimumSNR
    if np.isfinite(row['SNR']): snr=max([minimumSNR,row['SNR']]) # if we demand a higher SNR, great, but otherwise use minimum SNR
    conditions['SNR']=snr
    obj['conditions']=conditions
    objectiveList.append(obj)
    

### Perform the observations and assessment
We first compute the SNR and then the angular resolution

In [ ]:
import importlib
importlib.reload(metrics)
importlib.reload(sources)
addOldRes=False
for mission in missions:
    print('\nProcessing mission',mission['label'],'\n')
    obs_versions=[]
    if addOldRes: versions=[False,True]
    else: versions=[False]
    for useOldRes in versions:
        obs = list()
        for s in objectiveList:
            print('\nProcessing:',s.get('label','--unnamed--'))
            #print(s)
            if useOldRes:
                sgn=-1;
                if s['type']=='CW': sgn=1 #There is currently a pathology with getSourceSnr, it needs T>0 for CW and T<0 for chirp
                ob = metrics.getSourceSnr(s,mission,sgn*4.0*constants.year)
                ob = metrics.getResolution(ob)
            else:
                #print('OLD')
                #ob = metrics.getSourceSnr(s,mission,-4.0*constants.year)
                #print('NEW')
                #print('source:',s)
                ob = metrics.getSNRandSkyResolution(s,mission,Tmax=4)
            print('SNR,res',ob['SNR'],ob['Angular Resolution'])
            #Perform the assessment
            #The assessment is based on the ratio of the estimated capability to the objective for each of
            #resolution and SNR, with <=1 meaning that the objective is met
            #A total is based on the worst of those present
            conditions=s['conditions']
            assessment={}
            alleps=[]
            print("conditions are: ", conditions)
            if 'SNR' in conditions:
                #eps=ob['SNR']/conditions['SNR']
                eps=conditions['SNR']/ob['SNR']
                assessment['eps_SNR']=eps
                alleps.append(eps)
            if 'resolution' in conditions:
                #eps=conditions['resolution']/ob['Angular Resolution']
                eps=ob['Angular Resolution']/conditions['resolution']
                #print(ob['Angular Resolution'][-1],conditions['resolution'],eps)
                assessment['eps_res']=eps
                alleps.append(eps)
            #print('alleps',alleps)
            if len(alleps)>0: 
                assessment['eps']=max(alleps)*(1+sum(alleps)*0) #The last factor ensures that NANs stay NANs
            ob['assessment']=assessment    
            print("assessment is: ",assessment)
            obs.append(ob)
        obs_versions.append(obs)
        
    mission['obs']=obs_versions[0]
    if addOldRes:mission['obs_old']=obs_versions[1]



### Show assessments

In [ ]:
#Make a dataframe with the results
objectiveNames=df['provisional POC label'].values
if True:
    #by hand list of those which were not used in the STM as of 9/12/2022
    unused=['1.1.b', '1.2.b', '1.3.b', '2.2.b', '3.1.a', '3.2.a', '3.3.d', '4.1.a', '4.1.b', '5.1.a', '5.1.b', '5.1.c', '5.1.d', '5.2.a', '5.2.b', '5.3.a', '5.3.b', '5.4.a', '5.4.b', '5.5.b', '5.7.a']
    for i in range(len(objectiveNames)):
        oname=objectiveNames[i]
        if oname in unused: objectiveNames[i]='XX'+oname+'XX'
nm=len(missions)
colNames=missionNames
if addOldRes:nm=nm*2+1
data=np.zeros((len(obs),nm))
for i,mission in enumerate(missions):
    for j,ob in enumerate(mission['obs']):
        #print(missionNames[i],objectiveNames[j],ob['assessment'])
        temp = ob['assessment'].get('eps',None)
        #print('for observation ',ob['source'].get('label',None),' size of temp is',temp.size, ', shape is ', temp.shape, ', and ndim is ', temp.ndim)
        #data[j,i]=ob['assessment'].get('eps',None)
        data[j,i]=temp
if addOldRes:
    colNames=list(colNames)+['<--NEW | OLD-->']+[name+"'" for name in missionNames]
    data[:,len(missions)]=float('nan')
    for i,mission in enumerate(missions):
        for j,ob in enumerate(mission['obs_old']):
            #print(missionNames[i],objectiveNames[j],ob['assessment'])
            data[j,i+len(missions)+1]=ob['assessment'].get('eps',None)


adf = pd.DataFrame(data,
                  index=objectiveNames,
                  columns=colNames)

    
#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#pd.options.display.precision=2
s=adf.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink())
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Net performance")
s=s.format('{:.2g}')
display(s)
if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredObservations_net_'+dates+'.csv'
    #adf.to_csv(outfile,index=False)
    adf.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)

In [ ]:
#Make a dataframe with the results
objectiveNames=df['provisional POC label'].values
nm=len(missions)
colNames=missionNames
if addOldRes:nm=nm*2+1
data=np.zeros((len(obs),nm))
for i,mission in enumerate(missions):
    for j,ob in enumerate(mission['obs']):
        #print(missionNames[i],objectiveNames[j],ob['assessment'])
        temp = ob['assessment'].get('eps_SNR',None)
        #data[j,i]=ob['assessment'].get('eps_SNR',None)
        #print('type of temp is',type(temp))
        data[j,i]=temp
if addOldRes:
    colNames=list(colNames)+['<--NEW | OLD-->']+[name+"'" for name in missionNames]
    data[:,len(missions)]=float('nan')
    for i,mission in enumerate(missions):
        for j,ob in enumerate(mission['obs_old']):
            #print(missionNames[i],objectiveNames[j],ob['assessment'])
            data[j,i+len(missions)+1]=ob['assessment'].get('eps_SNR',None)


adf = pd.DataFrame(data,
                  index=objectiveNames,
                  columns=colNames)

    
#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#pd.options.display.precision=2
s=adf.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink())
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("SNR performance")
s=s.format('{:.2g}')
display(s)
if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredObservations_SNR_'+dates+'.csv'
    #adf.to_csv(outfile,index=False)
    adf.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)



In [ ]:
#Make a dataframe with the results
objectiveNames=df['provisional POC label'].values
nm=len(missions)
colNames=missionNames
if addOldRes:nm=nm*2+1
data=np.zeros((len(obs),nm))
for i,mission in enumerate(missions):
    for j,ob in enumerate(mission['obs']):
        #print(missionNames[i],objectiveNames[j],ob['assessment'])
        temp = ob['assessment'].get('eps_res',None)
        #print("the type of temp is ",type(temp))
        #data[j,i]=ob['assessment'].get('eps_res',None)
        data[j,i]=temp
if addOldRes:
    colNames=list(colNames)+['<--NEW | OLD-->']+[name+"'" for name in missionNames]
    data[:,len(missions)]=float('nan')
    for i,mission in enumerate(missions):
        for j,ob in enumerate(mission['obs_old']):
            #print(missionNames[i],objectiveNames[j],ob['assessment'])
            data[j,i+len(missions)+1]=ob['assessment'].get('eps_res',None)


adf = pd.DataFrame(data,
                  index=objectiveNames,
                  columns=colNames)

    
#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
#pd.options.display.precision=2
s=adf.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink())
s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Resolution performance")
s=s.format('{:.2g}')
display(s)
if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredObservations_res_'+dates+'.csv'
    adf.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)